# Experiment #2: Open vSwitch 4 Networks 2 Bridge Nodes 1 Shared Link

In this experiment we try to connect 4 networks with different hosts together with a bridge in the middle. We will be using Open vSwitch for the bridge, alternate solutions are Linux Bridge and Behavioral Model (BMv2).

## References
- [Open vSwitch Example](https://github.com/fabric-testbed/jupyter-examples/blob/main/fabric_examples/complex_recipes/openvswitch/openvswitch.ipynb)
- [BMv4](https://github.com/p4lang/behavioral-model)

## TODO's

- Add Open vSwitch flow configuratation on ports and so on.
- Add tools on how to track packets in the bridge.
- measure of congestion.


## Topology


<img src="./fig/Testbed-openvswitch-2.png" width=90%>

## Code

In [1]:
from ipaddress import ip_address, IPv4Address, IPv4Network
import ipaddress
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager


fablib = fablib_manager()
fablib.show_config();

Orchestrator,orchestrator.fabric-testbed.net
Credential Manager,cm.fabric-testbed.net
Core API,uis.fabric-testbed.net
Artifact Manager,artifacts.fabric-testbed.net
Token File,/home/alphani/.fabric_token.json
Project ID,220c2d8a-ae05-465a-a98b-8178415e1b30
Bastion Host,bastion.fabric-testbed.net
Bastion Username,sa2994_0000240697
Bastion Private Key File,/home/alphani/work/fabric_config/fabric_bastion_key
Slice Public Key File,/home/alphani/work/fabric_config/slice_key.pub
Slice Private Key File,/home/alphani/work/fabric_config/slice_key


In [2]:
slice_name= "Testbed-openvswitch-2"

sites = [site1] = fablib.get_random_sites(count=1)
print(f"Sites: {sites}")

site_node_count = 2
bridge1_name = 'bridge1'
bridge2_name = 'bridge2'

Sites: ['GPN']


In [3]:
slice = fablib.new_slice(name=slice_name)
default_image= 'default_ubuntu_22'

In [4]:
bridge1 = slice.add_node(name=bridge1_name, site=site1, cores=2, ram=8, disk=10, image=default_image, host=f"{site1.lower()}-w1.fabric-testbed.net")
bridge1_nic1 = bridge1.add_component(model='NIC_Basic', name='nic_local_1')
bridge1_nic2 = bridge1.add_component(model='NIC_Basic', name='nic_local_2')
bridge1_nic3 = bridge1.add_component(model='NIC_Basic', name='nic_local_3')

bridge2 = slice.add_node(name=bridge2_name, site=site1, cores=2, ram=8, disk=10, image=default_image, host=f"{site1.lower()}-w1.fabric-testbed.net")
bridge2_nic1 = bridge2.add_component(model='NIC_Basic', name='nic_local_1')
bridge2_nic2 = bridge2.add_component(model='NIC_Basic', name='nic_local_2')
bridge2_nic3 = bridge2.add_component(model='NIC_Basic', name='nic_local_3')

### Connecting bridges nodes 

In [5]:
net = slice.add_l2network(name=f'net-br1-br2')
net.add_interface(bridge1.get_interfaces()[2])
net.add_interface(bridge2.get_interfaces()[2])

### Adding Nodes to bridge 1

In [6]:
for br_num in range(2):
    print(f"Adding nodes to {site1}-bridge{br_num+1}")
    for node_num in range(site_node_count):
        node_name = f"{site1.lower()}-bridge{br_num}-{node_num+1}"
        node = slice.add_node(name=node_name, site=site1, cores=2, ram=8, disk=10, image=default_image, host=f"{site1.lower()}-w2.fabric-testbed.net")
        iface = node.add_component(model='NIC_Basic', name='nic_local').get_interfaces()[0]    
        net = slice.add_l2network(name=f"net-br{br_num}-{node_num+1}")
        net.add_interface(iface)

        if br_num%2==0:
            net.add_interface(bridge1.get_interfaces()[node_num])
        else:
            net.add_interface(bridge2.get_interfaces()[node_num])

Adding nodes to GPN-bridge1
Adding nodes to GPN-bridge2


In [7]:
slice.submit();


Retry: 11, Time: 325 sec


ID,eb7d7d3f-f2e2-4f6b-b762-64c9937677c7
Name,Testbed-openvswitch-2
Lease Expiration (UTC),2025-01-04 16:35:11 +0000
Lease Start (UTC),2025-01-03 16:35:11 +0000
Project ID,220c2d8a-ae05-465a-a98b-8178415e1b30
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
fba723b5-d8ba-4a9b-9a47-06cbf2b198a9,bridge1,2,8,10,default_ubuntu_22,qcow2,gpn-w1.fabric-testbed.net,GPN,ubuntu,2610:e0:a04c:fab2:f816:3eff:fe9c:950e,Active,,ssh -i /home/alphani/work/fabric_config/slice_key -F /home/alphani/work/fabric_config/ssh_config ubuntu@2610:e0:a04c:fab2:f816:3eff:fe9c:950e,/home/alphani/work/fabric_config/slice_key.pub,/home/alphani/work/fabric_config/slice_key
e0bb2a4e-2adc-4d80-b205-408e819389b1,bridge2,2,8,10,default_ubuntu_22,qcow2,gpn-w1.fabric-testbed.net,GPN,ubuntu,2610:e0:a04c:fab2:f816:3eff:fe82:2abb,Active,,ssh -i /home/alphani/work/fabric_config/slice_key -F /home/alphani/work/fabric_config/ssh_config ubuntu@2610:e0:a04c:fab2:f816:3eff:fe82:2abb,/home/alphani/work/fabric_config/slice_key.pub,/home/alphani/work/fabric_config/slice_key
971c5357-93f7-4b8f-81ff-e29db3deadec,gpn-bridge0-1,2,8,10,default_ubuntu_22,qcow2,gpn-w2.fabric-testbed.net,GPN,ubuntu,2610:e0:a04c:fab2:f816:3eff:feec:bde9,Active,,ssh -i /home/alphani/work/fabric_config/slice_key -F /home/alphani/work/fabric_config/ssh_config ubuntu@2610:e0:a04c:fab2:f816:3eff:feec:bde9,/home/alphani/work/fabric_config/slice_key.pub,/home/alphani/work/fabric_config/slice_key
509b86ef-16ff-4000-acb5-28c954ebcfbc,gpn-bridge0-2,2,8,10,default_ubuntu_22,qcow2,gpn-w2.fabric-testbed.net,GPN,ubuntu,2610:e0:a04c:fab2:f816:3eff:febd:5e20,Active,,ssh -i /home/alphani/work/fabric_config/slice_key -F /home/alphani/work/fabric_config/ssh_config ubuntu@2610:e0:a04c:fab2:f816:3eff:febd:5e20,/home/alphani/work/fabric_config/slice_key.pub,/home/alphani/work/fabric_config/slice_key
449cdcec-9090-411e-a482-1e6553d02957,gpn-bridge1-1,2,8,10,default_ubuntu_22,qcow2,gpn-w2.fabric-testbed.net,GPN,ubuntu,2610:e0:a04c:fab2:f816:3eff:fe55:e448,Active,,ssh -i /home/alphani/work/fabric_config/slice_key -F /home/alphani/work/fabric_config/ssh_config ubuntu@2610:e0:a04c:fab2:f816:3eff:fe55:e448,/home/alphani/work/fabric_config/slice_key.pub,/home/alphani/work/fabric_config/slice_key
e5fc3d4a-d655-44dd-91f4-5ebabd51c611,gpn-bridge1-2,2,8,10,default_ubuntu_22,qcow2,gpn-w2.fabric-testbed.net,GPN,ubuntu,2610:e0:a04c:fab2:f816:3eff:fe81:b8f9,Active,,ssh -i /home/alphani/work/fabric_config/slice_key -F /home/alphani/work/fabric_config/ssh_config ubuntu@2610:e0:a04c:fab2:f816:3eff:fe81:b8f9,/home/alphani/work/fabric_config/slice_key.pub,/home/alphani/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
b0674d8b-a11f-4211-978a-766f8755c845,net-br0-1,L2,L2Bridge,GPN,None,None,Active,
6bb81042-8c28-4f10-9d67-27afe579a192,net-br0-2,L2,L2Bridge,GPN,None,None,Active,
66fbd0b9-14d0-4bbc-a4a3-e27de4129d70,net-br1-1,L2,L2Bridge,GPN,None,None,Active,
4030eb84-f5fe-424c-9350-d9f01d0353ab,net-br1-2,L2,L2Bridge,GPN,None,None,Active,
85da2148-9988-4270-bcfc-56e49bd58d2c,net-br1-br2,L2,L2Bridge,GPN,None,None,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node,Switch Port
bridge1-nic_local_3-p1,p1,bridge1,net-br0-1,100,config,,06:7E:61:21:AF:20,enp9s0,enp9s0,fe80::47e:61ff:fe21:af20,6,HundredGigE0/0/0/5
bridge1-nic_local_1-p1,p1,bridge1,net-br0-2,100,config,,02:19:1B:38:14:52,enp7s0,enp7s0,fe80::19:1bff:fe38:1452,6,HundredGigE0/0/0/5
bridge1-nic_local_2-p1,p1,bridge1,net-br1-br2,100,config,,02:31:E6:AF:DC:68,enp8s0,enp8s0,fe80::31:e6ff:feaf:dc68,6,HundredGigE0/0/0/5
bridge2-nic_local_2-p1,p1,bridge2,net-br1-br2,100,config,,06:96:8A:E0:E7:8D,enp7s0,enp7s0,fe80::496:8aff:fee0:e78d,6,HundredGigE0/0/0/5
bridge2-nic_local_3-p1,p1,bridge2,net-br1-1,100,config,,0A:91:70:D5:AA:B0,enp9s0,enp9s0,fe80::891:70ff:fed5:aab0,6,HundredGigE0/0/0/5
bridge2-nic_local_1-p1,p1,bridge2,net-br1-2,100,config,,0A:82:C3:41:E6:C8,enp8s0,enp8s0,fe80::882:c3ff:fe41:e6c8,6,HundredGigE0/0/0/5
gpn-bridge0-1-nic_local-p1,p1,gpn-bridge0-1,net-br0-1,100,config,,02:0F:7E:EA:E5:ED,enp7s0,enp7s0,fe80::f:7eff:feea:e5ed,6,HundredGigE0/0/0/7
gpn-bridge0-2-nic_local-p1,p1,gpn-bridge0-2,net-br0-2,100,config,,02:49:03:96:49:74,enp7s0,enp7s0,fe80::49:3ff:fe96:4974,6,HundredGigE0/0/0/7
gpn-bridge1-1-nic_local-p1,p1,gpn-bridge1-1,net-br1-1,100,config,,02:E3:8F:3F:8C:66,enp7s0,enp7s0,fe80::e3:8fff:fe3f:8c66,6,HundredGigE0/0/0/7
gpn-bridge1-2-nic_local-p1,p1,gpn-bridge1-2,net-br1-2,100,config,,06:C8:27:0D:7F:02,enp7s0,enp7s0,fe80::4c8:27ff:fe0d:7f02,6,HundredGigE0/0/0/7



Time to print interfaces 353 seconds


# OVS 

In [8]:
try:
    for node in slice.get_nodes():
        if node.get_name().startswith("bridge"):
            stdout, stderr = node.execute('yes | sudo apt-get -y update && sudo apt-get upgrade', quiet=True) 
            stdout, stderr = node.execute('yes | sudo apt-get -y install openvswitch-switch openvswitch-common', quiet=True)
            stdout, stderr = node.execute('sudo apt-get -y install net-tools', quiet=True)
            print(f"done bridge: {node.get_name()}")
    print("Done")
except Exception as e:
    print(f"Exception: {e}")

done bridge: bridge1
done bridge: bridge2
Done


In [9]:
bridge1 = slice.get_node(name=bridge1_name)
stdout, stderr = bridge1.execute('sudo ovs-vsctl add-br br0')
for interface in bridge1.get_interfaces():
    stdout, stderr = bridge1.execute(f'sudo ovs-vsctl add-port br0 {interface.get_physical_os_interface_name()}')
    #Remove IP addresses for all interfaces
    stdout, stderr = bridge1.execute(f'sudo ifconfig {interface.get_physical_os_interface_name()} 0')
    
#bring the bridge up
stdout, stderr = bridge1.execute('sudo ifconfig br0 up')

print("Done")
stdout, stderr = bridge1.execute('sudo ovs-vsctl set bridge br0 stp_enable=true')
stdout, stderr = bridge1.execute('sudo ovs-appctl stp/show')


Done
---- br0 ----
Root ID:
  stp-priority  32768
  stp-system-id   32:ee:63:98:71:44
  stp-hello-time  2s
  stp-max-age     20s
  stp-fwd-delay   15s
  This bridge is the root

Bridge ID:
  stp-priority  32768
  stp-system-id   32:ee:63:98:71:44
  stp-hello-time  2s
  stp-max-age     20s
  stp-fwd-delay   15s

  Interface  Role       State      Cost  Pri.Nbr 
  ---------- ---------- ---------- ----- -------
  enp8s0     designated listening  2     128.1
  enp7s0     designated listening  2     128.2
  enp9s0     designated listening  2     128.3



In [10]:
bridge2 = slice.get_node(name=bridge2_name)
stdout, stderr = bridge2.execute('sudo ovs-vsctl add-br br0')
for interface in bridge2.get_interfaces():
    stdout, stderr = bridge2.execute(f'sudo ovs-vsctl add-port br0 {interface.get_physical_os_interface_name()}')
    #Remove IP addresses for all interfaces
    stdout, stderr = bridge2.execute(f'sudo ifconfig {interface.get_physical_os_interface_name()} 0')
    
#bring the bridge up
stdout, stderr = bridge2.execute('sudo ifconfig br0 up')

print("Done")
stdout, stderr = bridge2.execute('sudo ovs-vsctl set bridge br0 stp_enable=true')
stdout, stderr = bridge2.execute('sudo ovs-appctl stp/show')


Done
---- br0 ----
Root ID:
  stp-priority  32768
  stp-system-id   2a:63:96:a6:75:4d
  stp-hello-time  2s
  stp-max-age     20s
  stp-fwd-delay   15s
  This bridge is the root

Bridge ID:
  stp-priority  32768
  stp-system-id   2a:63:96:a6:75:4d
  stp-hello-time  2s
  stp-max-age     20s
  stp-fwd-delay   15s

  Interface  Role       State      Cost  Pri.Nbr 
  ---------- ---------- ---------- ----- -------
  enp8s0     designated listening  2     128.1
  enp7s0     designated listening  2     128.2
  enp9s0     designated listening  2     128.3



# Host Setup 

In [16]:
networks = [
    "10.10.10.1/24",
    "10.10.10.2/24",
    "10.10.10.3/24",
    "10.10.10.4/24"
]

for br_num in range(2):
    for i in range(2):
        host = slice.get_node(name=f'{site1.lower()}-bridge{br_num}-{i+1}')
        stdout, stderr = host.execute('sudo apt-get -y install net-tools', quiet=True)
        stdout, stderr = host.execute(f'sudo ip link set dev {host.get_interfaces()[0].get_physical_os_interface_name()} up', quiet=True)
        stdout, stderr = host.execute(f'sudo ip addr add {networks[i+2*br_num]} dev {host.get_interfaces()[0].get_physical_os_interface_name()}', quiet=True)


In [17]:
host1 = slice.get_node(name=f'{site1.lower()}-bridge0-1')
stdout, stderr = host1.execute('ping 10.10.10.2 -c 5')

PING 10.10.10.2 (10.10.10.2) 56(84) bytes of data.
64 bytes from 10.10.10.2: icmp_seq=1 ttl=64 time=0.288 ms
64 bytes from 10.10.10.2: icmp_seq=2 ttl=64 time=0.166 ms
64 bytes from 10.10.10.2: icmp_seq=3 ttl=64 time=0.279 ms
64 bytes from 10.10.10.2: icmp_seq=4 ttl=64 time=0.147 ms
64 bytes from 10.10.10.2: icmp_seq=5 ttl=64 time=0.121 ms

--- 10.10.10.2 ping statistics ---
5 packets transmitted, 5 received, 0% packet loss, time 4100ms
rtt min/avg/max/mdev = 0.121/0.200/0.288/0.069 ms


In [21]:
stdout, stderr = host1.execute('ping 10.10.10.3 -c 5')

PING 10.10.10.3 (10.10.10.3) 56(84) bytes of data.
64 bytes from 10.10.10.3: icmp_seq=1 ttl=64 time=1.10 ms
64 bytes from 10.10.10.3: icmp_seq=2 ttl=64 time=0.182 ms
64 bytes from 10.10.10.3: icmp_seq=3 ttl=64 time=0.171 ms
64 bytes from 10.10.10.3: icmp_seq=4 ttl=64 time=0.164 ms
64 bytes from 10.10.10.3: icmp_seq=5 ttl=64 time=0.254 ms

--- 10.10.10.3 ping statistics ---
5 packets transmitted, 5 received, 0% packet loss, time 4053ms
rtt min/avg/max/mdev = 0.164/0.374/1.099/0.363 ms


In [19]:
stdout, stderr = host1.execute('ping 10.10.10.4 -c 5')

PING 10.10.10.4 (10.10.10.4) 56(84) bytes of data.
64 bytes from 10.10.10.4: icmp_seq=1 ttl=64 time=0.804 ms
64 bytes from 10.10.10.4: icmp_seq=2 ttl=64 time=0.242 ms
64 bytes from 10.10.10.4: icmp_seq=3 ttl=64 time=0.168 ms
64 bytes from 10.10.10.4: icmp_seq=4 ttl=64 time=0.181 ms
64 bytes from 10.10.10.4: icmp_seq=5 ttl=64 time=0.167 ms

--- 10.10.10.4 ping statistics ---
5 packets transmitted, 5 received, 0% packet loss, time 4074ms
rtt min/avg/max/mdev = 0.167/0.312/0.804/0.247 ms


In [ ]:
slice.delete()